# BERT

In [1]:
label_to_class= "offensive.language"  # enter the label to be classified
version = "E" # enter the version

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/hate_speech/models

/content/drive/MyDrive/hate_speech/models


In [4]:
! pip install transformers==4.26.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.1 MB/s eta 0:00:00


In [5]:
import torch.nn as nn
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import pandas as pd
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
import time
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback

from collections import Counter
     

In [6]:
# Define pretrained tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [7]:
# no drops
# csv = pd.read_csv('../data/version'+version+'_train.csv',header=0)

# # create a new csv df
# csv_new = pd.DataFrame(csv, columns=[label_to_class, "tweet_hashed"])
# # drop all rows that have any NaN values
# csv_new_clean = csv_new.dropna(axis=0,how="any")

# drops to 3 examples per tweet
csv = pd.read_csv('../data/version'+version+'_train.csv',header=0)
# create a new csv df
csv_new = pd.DataFrame(csv, columns=[label_to_class, "tweet_hashed"])
# drop all rows that have any NaN values
csv_new_clean = csv_new.dropna(axis=0,how="any")
# save tweets into a list
tweet=list(csv_new_clean['tweet_hashed'])
# count tweet freqs
tweet_count=Counter(tweet)
for t, c in tweet_count.items():
    if c > 3:
        # get the index for a specific tweet into a list
        index = csv_new_clean[csv_new_clean.tweet_hashed == t].index.tolist()
        # randomly choose index to drop
        index_to_drop = random.sample(index, c-3)
        csv_new_clean = csv_new_clean.drop(index_to_drop, axis=0)

In [8]:
# calculate max length of the tweets
X= list(csv_new_clean["tweet_hashed"])
max_length = 0
for x in X:
    ids = tokenizer.encode(x)
    max_length = max(len(ids),max_length)

torch.save(max_length, "bert.max_length."+label_to_class+version)

y = list(csv_new_clean[label_to_class])
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=max_length)
X_dev_tokenized = tokenizer(X_dev, padding=True, truncation=True, max_length=max_length)

In [9]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx]).long()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [10]:
train_dataset = Dataset(X_train_tokenized, y_train)
dev_dataset = Dataset(X_dev_tokenized, y_dev)

In [11]:
# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    #labels= np.argmax(labels, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average = "weighted")
    precision = precision_score(y_true=labels, y_pred=pred, average = "weighted")
    f1 = f1_score(y_true=labels, y_pred=pred, average = "weighted")

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [12]:
# Define Trainer

args = TrainingArguments(
    output_dir=label_to_class+version,
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    seed=42,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)

In [ ]:
# Train pre-trained model
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5397
  Num Epochs = 15
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10125
  Number of trainable parameters = 109483778


Step,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 1350
  Batch size = 8


# test the models

In [ ]:

test_data_version = "E"

# init test file
test_csv = pd.read_csv('../data/version'+test_data_version+'_test.csv', header=0)

# create a new csv df with all the original columns
test_csv_new = pd.DataFrame(test_csv, columns=["id", "version","batch.tweet","tweet.id", "tweet_hashed", "hate.speech", "offensive.language"])


In [ ]:
def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

for label_to_test in ["hate.speech", "offensive.language"]:
  for version_to_test in ["A","B","C","D","E"]:

    # preprocess the test data
    max_length = torch.load("bert.max_length."+label_to_test+version_to_test, map_location=DEVICE)

    X_test = tokenizer(list(test_csv_new["tweet_hashed"]), padding=True, truncation=True, max_length=max_length)

    test_dataset = Dataset(X_test)

    # Load trained model
    model_path = "bert.model."+label_to_test+version_to_test
    trained_model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

    # Define tester
    args = TrainingArguments(
        output_dir="tester",
        dataloader_pin_memory=False,
    )
    tester = Trainer(model=trained_model, args=args)

    # Make prediction
    raw_pred, _, _ = tester.predict(test_dataset)
    
    y_pred_scores = []
    for i in raw_pred:
      y_pred_scores.append(softmax(i)[1])

    # Preprocess raw predictions
    y_pred = np.argmax(raw_pred, axis=1)

    if label_to_test == "hate.speech":
      if version_to_test == "A":
        column = 7
      elif version_to_test == "B":
        column = 9
      elif version_to_test == "C":
        column = 11
      elif version_to_test == "D":
        column = 13
      elif version_to_test == "E":
        column = 15
      else:
        raise KeyError
    elif label_to_test == "offensive.language":
      if version_to_test == "A":
        column = 17
      elif version_to_test == "B":
        column = 19
      elif version_to_test == "C":
        column = 21
      elif version_to_test == "D":
        column = 23
      elif version_to_test == "E":
        column = 25
      else:
        raise KeyError
    else:
      raise KeyError

    test_csv_new.insert(column,label_to_test+"_preds_"+version_to_test,y_pred)
    test_csv_new.insert(column+1,label_to_test+"_preds_"+version_to_test+"_scores",y_pred_scores)

test_csv_new

loading configuration file bert.model.hate.speechA/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.hate.speechA/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenc

loading configuration file bert.model.hate.speechB/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.hate.speechB/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenc

loading configuration file bert.model.hate.speechC/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.hate.speechC/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenc

loading configuration file bert.model.hate.speechD/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.hate.speechD/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenc

loading configuration file bert.model.hate.speechE/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.hate.speechE/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenc

...

loading configuration file bert.model.offensive.languageB/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.offensive.languageB/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of 

loading configuration file bert.model.offensive.languageC/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.offensive.languageC/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of 

loading configuration file bert.model.offensive.languageD/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.offensive.languageD/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of 

loading configuration file bert.model.offensive.languageE/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.offensive.languageE/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of 

,id,version,batch.tweet,tweet.id,tweet_hashed,hate.speech,offensive.language,hate.speech_preds_A,hate.speech_preds_A_scores,hate.speech_preds_B,...,offensive.language_preds_A,offensive.language_preds_A_scores,offensive.language_preds_B,offensive.language_preds_B_scores,offensive.language_preds_C,offensive.language_preds_C_scores,offensive.language_preds_D,offensive.language_preds_D_scores,offensive.language_preds_E,offensive.language_preds_E_scores
0,125,E,R1,1,@###### bro that hoe live,1.0,0,0,0.131821,0,...,1,0.757343,1,0.911581,1,0.949679,1,0.874263,1,0.899440
1,173,E,R1,1,@###### bro that hoe live,0.0,1,0,0.131821,0,...,1,0.757343,1,0.911581,1,0.949679,1,0.874263,1,0.899440
2,219,E,R1,1,@###### bro that hoe live,0.0,1,0,0.131821,0,...,1,0.757343,1,0.911581,1,0.949679,1,0.874263,1,0.899440
3,223,E,R1,1,@###### bro that hoe live,0.0,1,0,0.131821,0,...,1,0.757343,1,0.911581,1,0.949679,1,0.874263,1,0.899440
4,228,E,R1,1,@###### bro that hoe live,0.0,1,0,0.131821,0,...,1,0.757343,1,0.911581,1,0.949679,1,0.874263,1,0.899440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3330,996,E,R49,2999,RT @###### My favorite episode of Friends is t...,0.0,1,0,0.266801,1,...,0,0.281548,0,0.103202,0,0.144670,0,0.480707,1,0.757307
3331,1073,E,R49,2999,RT @###### My favorite episode of Friends is t...,0.0,0,0,0.266801,1,...,0,0.281548,0,0.103202,0,0.144670,0,0.480707,1,0.757307
3332,1472,E,R49,2999,RT @###### My favorite episode of Friends is t...,0.0,0,0,0.266801,1,...,0,0.281548,0,0.103202,0,0.144670,0,0.480707,1,0.757307
3333,1481,E,R49,2999,RT @###### My favorite episode of Friends is t...,0.0,0,0,0.266801,1,...,0,0.281548,0,0.103202,0,0.144670,0,0.480707,1,0.757307


In [ ]:
test_csv_new.to_csv("../preds/bert_test"+test_data_version+".csv")